## Imports

In [1]:
import requests
from bs4 import BeautifulSoup

In [1]:
# Lendo a primeira página

In [87]:
embrapa_url = "http://vitibrasil.cnpuv.embrapa.br/index.php"
re = requests.get(embrapa_url)
soup = BeautifulSoup(re._content)
# páginas
pages = {}
for i in soup.find_all('button'):
    texto = i.text
    value = '' if 'value' not in i.attrs.keys() else i.attrs['value']
    pages[texto] = {
        'link': f'{embrapa_url}?opcao={value}',
        'has_subpages': 'No', 
        'sub_pages': {},
        'download_link': {}
        }
    print(texto, value)

Apresentação opt_01
Produção opt_02
Processamento opt_03
Comercialização opt_04
Importação opt_05
Exportação opt_06
Publicação opt_07


In [96]:
# Sub Páginas

for key, values in pages.items():
    print(key)
    re = requests.get(values['link'])
    soup = BeautifulSoup(re._content)
    subpages_dict = {}
    for i in soup.find_all('button'):
        text = i.text
        btn_value = '' if 'value' not in i.attrs.keys() else i.attrs['value']
        if 'subopt' in btn_value:
            print(f"subopt: {btn_value}")
            values['has_subpages'] = 'yes' 
            subpages_dict[text] = f"{values['link']}&subopcao={btn_value}"
    values['sub_pages'] = subpages_dict


Apresentação
Produção
Processamento
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
Comercialização
Importação
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
subopt: subopt_05
Exportação
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
Publicação


In [108]:
# Download Links
for key, values in pages.items():
    if key in ['Apresentação', 'Publicação']: continue
    if values['has_subpages'] == 'No':
        re = requests.get(values['link'])
        soup = BeautifulSoup(re._content)
        for link_element in soup.find_all('a', class_='footer_content'):
            if 'href' in link_element.attrs.keys() and 'DOWNLOAD' in link_element.text:
                values['download_link'] = [embrapa_url.replace('index.php', link_element['href'])]
                # Fazendo a requisição para baixar o conteúdo do CSV
                if 'pdf' in link_element['href']: continue
                response = requests.get(embrapa_url.replace('index.php', link_element['href']))
                # Checando se o download foi bem-sucedido
                if response.status_code == 200:
                    # Salvando o conteúdo do arquivo
                    name = link_element['href'].split('/')[-1]
                    with open(f"data/data_{name}", "wb") as file:
                        file.write(response.content)
                    print("Arquivo CSV baixado e salvo com sucesso.")
                else:
                    print(f"Erro ao baixar o arquivo: {response.status_code}")
    else:
        download_link = []
        for subpages in values['sub_pages'].values():
            re = requests.get(subpages)
            soup = BeautifulSoup(re._content)
            for link_element in soup.find_all('a', class_='footer_content'):
                if 'href' in link_element.attrs.keys() and 'DOWNLOAD' in link_element.text:
                    download_link.append(embrapa_url.replace('index.php', link_element['href']))
                    # Fazendo a requisição para baixar o conteúdo do CSV
                    if 'pdf' in link_element['href']: continue
                    response = requests.get(embrapa_url.replace('index.php', link_element['href']))
                    # Checando se o download foi bem-sucedido
                    if response.status_code == 200:
                        # Salvando o conteúdo do arquivo
                        name = link_element['href'].split('/')[-1]
                        with open(f"data/data_{name}", "wb") as file:
                            file.write(response.content)
                        print("Arquivo CSV baixado e salvo com sucesso.")
                    else:
                        print(f"Erro ao baixar o arquivo: {response.status_code}")
        values['download_link'] = download_link



Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
